## Transfer scraped recipe pages from s3 bucket, load to recipe_pages_coll 

In [21]:
import pymongo 
import boto3 
import os 
import cPickle

In [5]:
os.environ.keys()

['GIT_PS1_SHOWDIRTYSTATE',
 'SPARK_HOME',
 'TERM_PROGRAM_VERSION',
 'LOGNAME',
 'USER',
 'HOME',
 'PATH',
 'PS1',
 'LD_LIBRARY_PATH',
 'TERM_PROGRAM',
 'LANG',
 'TERM',
 'SHELL',
 'SHLVL',
 'SECURITYSESSIONID',
 'XPC_FLAGS',
 'DISPLAY',
 '_',
 'ITERM_ORIG_PS1',
 'TERM_SESSION_ID',
 'XPC_SERVICE_NAME',
 'JPY_PARENT_PID',
 'PYTHONPATH',
 'SSH_AUTH_SOCK',
 'Apple_PubSub_Socket_Render',
 'GIT_PAGER',
 'TMPDIR',
 'AWS_ACCESS_KEY',
 'MPLBACKEND',
 'PAGER',
 'CLICOLOR',
 '__CF_USER_TEXT_ENCODING',
 'PWD',
 'AWS_SECRET_ACCESS_KEY',
 'HADOOP_HOME',
 'ITERM_PREV_PS1']

In [6]:
AWS_KEY = os.environ['AWS_ACCESS_KEY']
AWS_SECRET = os.environ['AWS_SECRET_ACCESS_KEY']

In [10]:
session = boto3.Session(aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)
s3 = session.resource('s3')

mybucket = s3.Bucket('ohailolcat')

In [12]:
recipe_page_files = [a.key for a in mybucket.objects.filter(Prefix='recipe_pages/')]

In [13]:
print len(recipe_page_files)

2089


In [16]:
sum([a.endswith('.pkl') for a in recipe_page_files])

2089

In [17]:
recipe_page_files[0]

u'recipe_pages/10004_pages.pkl'

In [18]:
reclist = [a.key.split('/')[1].split('_')[0] for a in mybucket.objects.filter(Prefix='recipe_pages/')]

In [24]:
rec_page = [a for a in mybucket.objects.filter(Prefix='recipe_pages/')]

In [26]:
pg = cPickle.loads(rec_page[0].get()['Body'].read())

In [29]:
pg.keys()

[u'10004_page', 'member_ID']

In [32]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
allrecipes_db = client.allrecipes 
recipe_pages_coll = allrecipes_db.recipe_pages 

In [33]:
recipe_pages_coll.create_index([('recipe_ID',pymongo.ASCENDING)], unique=True)

u'recipe_ID_1'

In [28]:
pg.items()

[(u'10004_page',
  u'<!DOCTYPE html><html><head><style type="text/css">@charset "UTF-8";[ng\\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\\:form{display:block;}</style>\n    <title>Pavlova Recipe - Allrecipes.com</title>\n        <meta property="og:title" content="Pavlova Recipe">\n\n    <meta property="og:site_name" content="Allrecipes">\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n        <meta id="metaDescription" name="description" content="My grandmother\'s recipe from New Zealand for the famous Pavlova.">\n        <meta property="og:description" content="My grandmother\'s recipe from New Zealand for the famous Pavlova.">\n                <meta name="robots" content="noodp,noydir">\n\n    \n\n        <link id="canonicalUrl" rel="canonical" href="http://allrecipes.com/recipe/10004/pavlova/">\n        <meta property="og:url" content="http://

In [54]:
# All the keys are the same except `link` and `_id`, which is okay.  

def get_recipe_pages_from_s3(reclist=None): 
    """Given a list of recipe_IDs, populate recipe_pages_coll with data from files in 
    recipe_pages/"""
    if not reclist: 
        reclist = [a.key.split('/')[1].split('_')[0] for a in mybucket.objects.filter(Prefix='recipe_pages/')]
    print "number of recipe files in s3 recipe_pages:", len(reclist)
    print "number of docs in recipe_pages_coll", recipe_pages_coll.count()
    recipe_pages_list = [a['recipe_ID'] for a in recipe_pages_coll.find()]
    recipe_pages_list.extend(exceptions)
    # member_pages_coll.ensureIndex({name: 1, nodes: 1}, {unique: true, dropDups: true})
    # member_pages_coll.ensure_index({"member_ID": 1}, {unique:}) .... 
    # to_get = set(memlist) - set(member_pages_memlist)
    # print "files to get:", len(to_get)
    for i,a in enumerate(mybucket.objects.filter(Prefix='recipe_pages/')): 
        print "{}, accessing {}".format(i,a.key)
        if a.key.split('/')[1].split('_')[0] not in recipe_pages_list:
            recdata = cPickle.loads(a.get()['Body'].read())
            recdata_page = recdata[recdata['member_ID']+"_page"]
            print "{} Inserting: {}".format(i, recdata["member_ID"])
            recipe_pages_coll.insert_one({'recipe_ID': recdata['member_ID'], 
                                          'page':recdata_page})

In [55]:
get_recipe_pages_from_s3()

number of recipe files in s3 recipe_pages: 2089
number of docs in recipe_pages_coll 1405
0, accessing recipe_pages/10004_pages.pkl
1, accessing recipe_pages/100690_pages.pkl
2, accessing recipe_pages/100822_pages.pkl
3, accessing recipe_pages/100839_pages.pkl
4, accessing recipe_pages/10147_pages.pkl
5, accessing recipe_pages/10149_pages.pkl
6, accessing recipe_pages/101522_pages.pkl
7, accessing recipe_pages/101530_pages.pkl
8, accessing recipe_pages/101732_pages.pkl
9, accessing recipe_pages/10198_pages.pkl
10, accessing recipe_pages/10204_pages.pkl
11, accessing recipe_pages/102197_pages.pkl
12, accessing recipe_pages/10221_pages.pkl
13, accessing recipe_pages/102235_pages.pkl
14, accessing recipe_pages/10226_pages.pkl
15, accessing recipe_pages/10232_pages.pkl
16, accessing recipe_pages/10261_pages.pkl
17, accessing recipe_pages/103347_pages.pkl
18, accessing recipe_pages/10340_pages.pkl
19, accessing recipe_pages/10356_pages.pkl
20, accessing recipe_pages/103971_pages.pkl
21, acce

1000, accessing recipe_pages/232836_pages.pkl
1001, accessing recipe_pages/232890_pages.pkl
1002, accessing recipe_pages/232907_pages.pkl
1003, accessing recipe_pages/232914_pages.pkl
1004, accessing recipe_pages/233007_pages.pkl
1005, accessing recipe_pages/23309_pages.pkl
1006, accessing recipe_pages/233160_pages.pkl
1007, accessing recipe_pages/233165_pages.pkl
1008, accessing recipe_pages/233170_pages.pkl
1009, accessing recipe_pages/233226_pages.pkl
1010, accessing recipe_pages/233230_pages.pkl
1011, accessing recipe_pages/233270_pages.pkl
1012, accessing recipe_pages/233276_pages.pkl
1013, accessing recipe_pages/233289_pages.pkl
1014, accessing recipe_pages/233295_pages.pkl
1015, accessing recipe_pages/233299_pages.pkl
1016, accessing recipe_pages/233300_pages.pkl
1017, accessing recipe_pages/233345_pages.pkl
1018, accessing recipe_pages/233429_pages.pkl
1019, accessing recipe_pages/233431_pages.pkl
1020, accessing recipe_pages/233437_pages.pkl
1021, accessing recipe_pages/233504

1406 Inserting: 25413
1407, accessing recipe_pages/25415_pages.pkl
1407 Inserting: 25415
1408, accessing recipe_pages/254211_pages.pkl
1408 Inserting: 254211
1409, accessing recipe_pages/254244_pages.pkl
1409 Inserting: 254244
1410, accessing recipe_pages/25425_pages.pkl
1410 Inserting: 25425
1411, accessing recipe_pages/254280_pages.pkl
1411 Inserting: 254280
1412, accessing recipe_pages/254281_pages.pkl
1412 Inserting: 254281
1413, accessing recipe_pages/254294_pages.pkl
1413 Inserting: 254294
1414, accessing recipe_pages/25431_pages.pkl
1414 Inserting: 25431
1415, accessing recipe_pages/25432_pages.pkl
1415 Inserting: 25432
1416, accessing recipe_pages/25433_pages.pkl
1416 Inserting: 25433
1417, accessing recipe_pages/254392_pages.pkl
1417 Inserting: 254392
1418, accessing recipe_pages/254451_pages.pkl
1418 Inserting: 254451
1419, accessing recipe_pages/254470_pages.pkl
1419 Inserting: 254470
1420, accessing recipe_pages/25447_pages.pkl
1420 Inserting: 25447
1421, accessing recipe_p

1526 Inserting: 258454
1527, accessing recipe_pages/258455_pages.pkl
1527 Inserting: 258455
1528, accessing recipe_pages/258494_pages.pkl
1528 Inserting: 258494
1529, accessing recipe_pages/25898_pages.pkl
1529 Inserting: 25898
1530, accessing recipe_pages/25957_pages.pkl
1530 Inserting: 25957
1531, accessing recipe_pages/25958_pages.pkl
1531 Inserting: 25958
1532, accessing recipe_pages/25964_pages.pkl
1532 Inserting: 25964
1533, accessing recipe_pages/26028_pages.pkl
1533 Inserting: 26028
1534, accessing recipe_pages/26030_pages.pkl
1534 Inserting: 26030
1535, accessing recipe_pages/26047_pages.pkl
1535 Inserting: 26047
1536, accessing recipe_pages/26056_pages.pkl
1536 Inserting: 26056
1537, accessing recipe_pages/26183_pages.pkl
1537 Inserting: 26183
1538, accessing recipe_pages/26192_pages.pkl
1538 Inserting: 26192
1539, accessing recipe_pages/26257_pages.pkl
1539 Inserting: 26257
1540, accessing recipe_pages/26266_pages.pkl
1540 Inserting: 26266
1541, accessing recipe_pages/26293_

1649 Inserting: 50410
1650, accessing recipe_pages/50459_pages.pkl
1650 Inserting: 50459
1651, accessing recipe_pages/50896_pages.pkl
1651 Inserting: 50896
1652, accessing recipe_pages/51711_pages.pkl
1652 Inserting: 51711
1653, accessing recipe_pages/51850_pages.pkl
1653 Inserting: 51850
1654, accessing recipe_pages/52461_pages.pkl
1654 Inserting: 52461
1655, accessing recipe_pages/52547_pages.pkl
1655 Inserting: 52547
1656, accessing recipe_pages/52572_pages.pkl
1656 Inserting: 52572
1657, accessing recipe_pages/52635_pages.pkl
1657 Inserting: 52635
1658, accessing recipe_pages/52934_pages.pkl
1658 Inserting: 52934
1659, accessing recipe_pages/53086_pages.pkl
1659 Inserting: 53086
1660, accessing recipe_pages/53224_pages.pkl
1660 Inserting: 53224
1661, accessing recipe_pages/53304_pages.pkl
1661 Inserting: 53304
1662, accessing recipe_pages/53546_pages.pkl
1662 Inserting: 53546
1663, accessing recipe_pages/53560_pages.pkl
1663 Inserting: 53560
1664, accessing recipe_pages/53574_pages

1766 Inserting: 64441526
1767, accessing recipe_pages/64456822_pages.pkl
1767 Inserting: 64456822
1768, accessing recipe_pages/64464672_pages.pkl
1768 Inserting: 64464672
1769, accessing recipe_pages/64475940_pages.pkl
1769 Inserting: 64475940
1770, accessing recipe_pages/64475941_pages.pkl
1770 Inserting: 64475941
1771, accessing recipe_pages/64480295_pages.pkl
1771 Inserting: 64480295
1772, accessing recipe_pages/64482559_pages.pkl
1772 Inserting: 64482559
1773, accessing recipe_pages/64493472_pages.pkl
1773 Inserting: 64493472
1774, accessing recipe_pages/64499898_pages.pkl
1774 Inserting: 64499898
1775, accessing recipe_pages/64502572_pages.pkl
1775 Inserting: 64502572
1776, accessing recipe_pages/64504099_pages.pkl
1776 Inserting: 64504099
1777, accessing recipe_pages/64510099_pages.pkl
1777 Inserting: 64510099
1778, accessing recipe_pages/64511627_pages.pkl
1778 Inserting: 64511627
1779, accessing recipe_pages/64512772_pages.pkl
1779 Inserting: 64512772
1780, accessing recipe_pag

1883 Inserting: 7202
1884, accessing recipe_pages/72068_pages.pkl
1884 Inserting: 72068
1885, accessing recipe_pages/72126_pages.pkl
1885 Inserting: 72126
1886, accessing recipe_pages/7216_pages.pkl
1886 Inserting: 7216
1887, accessing recipe_pages/72194_pages.pkl
1887 Inserting: 72194
1888, accessing recipe_pages/72381_pages.pkl
1888 Inserting: 72381
1889, accessing recipe_pages/72427_pages.pkl
1889 Inserting: 72427
1890, accessing recipe_pages/72520_pages.pkl
1890 Inserting: 72520
1891, accessing recipe_pages/72631_pages.pkl
1891 Inserting: 72631
1892, accessing recipe_pages/7265_pages.pkl
1892 Inserting: 7265
1893, accessing recipe_pages/72662_pages.pkl
1893 Inserting: 72662
1894, accessing recipe_pages/72673_pages.pkl
1894 Inserting: 72673
1895, accessing recipe_pages/72878_pages.pkl
1895 Inserting: 72878
1896, accessing recipe_pages/7289_pages.pkl
1896 Inserting: 7289
1897, accessing recipe_pages/72985_pages.pkl
1897 Inserting: 72985
1898, accessing recipe_pages/73269_pages.pkl
18

2008, accessing recipe_pages/85517_pages.pkl
2008 Inserting: 85517
2009, accessing recipe_pages/85757_pages.pkl
2009 Inserting: 85757
2010, accessing recipe_pages/8614_pages.pkl
2010 Inserting: 8614
2011, accessing recipe_pages/8619_pages.pkl
2011 Inserting: 8619
2012, accessing recipe_pages/86245_pages.pkl
2012 Inserting: 86245
2013, accessing recipe_pages/86283_pages.pkl
2013 Inserting: 86283
2014, accessing recipe_pages/86362_pages.pkl
2014 Inserting: 86362
2015, accessing recipe_pages/86394_pages.pkl
2015 Inserting: 86394
2016, accessing recipe_pages/8640_pages.pkl
2016 Inserting: 8640
2017, accessing recipe_pages/86410_pages.pkl
2017 Inserting: 86410
2018, accessing recipe_pages/86416_pages.pkl
2018 Inserting: 86416
2019, accessing recipe_pages/8660_pages.pkl
2019 Inserting: 8660
2020, accessing recipe_pages/8662_pages.pkl
2020 Inserting: 8662
2021, accessing recipe_pages/86649_pages.pkl
2021 Inserting: 86649
2022, accessing recipe_pages/86695_pages.pkl
2022 Inserting: 86695
2023,

In [53]:
exceptions = ['25309', '253156']